In [1]:
import os
import numpy as np
import seaborn as sns
import pandas as pd
import sys
import random

from datetime import datetime, timedelta

from matplotlib import rcParams
import matplotlib.pyplot as plt

from sklearn.neighbors import NearestNeighbors
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

from IPython.display import display
import importlib

sys.path.append('../utils') 
import cleaning_utils as cu

importlib.reload(cu)

<module 'cleaning_utils' from 'c:\\Users\\34642\\OneDrive\\Escritorio\\Synthesis Project I\\Notebooks\\../utils\\cleaning_utils.py'>

### Environmental Configurations

In [2]:
pd.set_option('display.max_columns', None)

In [3]:
data_path = os.path.join("..", "Data")
schedules_df = pd.read_excel(os.path.join(data_path, "Schedules.xlsx"))
data_df = pd.read_excel(os.path.join(data_path, "Data.xlsx"))
clients_df = pd.read_excel(os.path.join(data_path, "Clients.xlsx"))
transl_cost_pairs_df = pd.read_excel(os.path.join(data_path, "TranslatorsCost+Pairs.xlsx"))

In [4]:
data_df['START'] = pd.to_datetime(data_df['START'], errors='coerce')

### Envirinmental Variables

In [5]:
RANDOM_SEED = 42

TRANSLATORS_UNAVAILABLE = []

wildcards = [None, "Quality", "Time", "Cost"]
task_types = data_df["TASK_TYPE"].unique()
unique_language_pairs = data_df[["SOURCE_LANG", "TARGET_LANG"]].drop_duplicates().reset_index(drop=True)

> **Note:** `TRANSLATORS_UNAVAILABLE` is a list for keeping track of translators alredy assigned or performing a task

In [6]:
## TODO we have to think what to do with translators that can perform multiple tasks

### Data Cleaning

In [7]:
# Function to detect invalid rows
data_df, start_invalid_dates = cu.drop_invalid_dates(data_df, 'START')
data_df, end_invalid_dates = cu.drop_invalid_dates(data_df, 'END')
data_df, delivered_invalid_dates = cu.drop_invalid_dates(data_df, 'DELIVERED')
data_df, delivered_invalid_dates = cu.drop_invalid_dates(data_df, 'ASSIGNED')

print(" ")

# Show the invalid dates in each column
print("Invalid START dates:\n", start_invalid_dates, "\n")
print("Invalid END dates:\n", end_invalid_dates, "\n")
print("Invalid DELIVERED dates:\n", delivered_invalid_dates, "\n")

Dropping invalid rows in column 'START'...
No invalid dates found in column 'END'.
No invalid dates found in column 'DELIVERED'.
No invalid dates found in column 'ASSIGNED'.
 
Invalid START dates:
 91127   NaT
Name: START, dtype: datetime64[ns] 

Invalid END dates:
 Series([], Name: END, dtype: datetime64[ns]) 

Invalid DELIVERED dates:
 Series([], Name: ASSIGNED, dtype: datetime64[ns]) 



In [8]:
data_df = cu.drop_invalid_rows(data_df)

Dropped 25764 rows with missing values or START >= END.


#### Task Information

- **PROJECT_ID**: Project code (additional info, likely not necessary).
- **PM**: Responsible management team.
- **TASK_ID**: Task code.
- **TRANSLATOR**: Translator responsible for the task.
-----
- **START**: Task start date.
- **END**: Theoretical task delivery date (can be compared with `DELIVERED` to check for delays).
-----
Some considerations must be taken into account:
- **DTP**: Desktop-Publishing tasks.
- **Engineering**: Engineering tasks such as file conversions, coding, etc.
- **LanguageLead**: Linguistic management tasks. Assigned to highly experienced and quality-oriented individuals who regularly work on the project.
- **Management**: General management tasks.
- **Miscellaneous**: Various linguistic tasks.
- **PostEditing**: Post-editing tasks. Similar to Translation tasks but with slightly different skills required for the TRANSLATOR.
- **ProofReading**: Full review of a Translation or PostEditing. Always follows a Translation or PostEditing. The TRANSLATOR assigned must have more 
experience than the person who performed the initial step.
- **Spotcheck**: Partial review of a Translation or PostEditing. Similar conditions as ProofReading.
- **TEST**: Test required to qualify for working with a client. Should be assigned to the most experienced and high-quality TRANSLATOR 
for the client or topic, regardless of price but considering the deadline.
- **Training**: Translator experience and quality are not considered.
- **Translation**: Translation task. The translator’s quality can be slightly lower if the ProofReading (not Spotcheck) is done by a superior. If 
Spotcheck is done, the required quality must be met.
-----
- **SOURCE_LANG**: Source language.
- **TARGET_LANG**: Target language.
-----
- **ASSIGNED**: Time when the task is assigned (pre-notice) to the TRANSLATOR.
- **READY**: Time when the TRANSLATOR is notified they can start.
- **WORKING**: Time when the TRANSLATOR starts the task.
- **DELIVERED**: Time when the TRANSLATOR delivers the task.
- **RECEIVED**: Time when the PM receives the task.
- **CLOSE**: Time when the PM marks the task as completed.
-----
- **FORECAST**: Estimated hours for completion.
- **HOURLY_RATE**: Task hourly rate.
- **COST**: Total task cost.
- **QUALITY_EVALUATION**: Quality control evaluation.
-----
- **MANUFACTURER**: Client.
- **MANUFACTURER_SECTOR**: Level 1 client categorization.
- **MANUFACTURER_INDUSTRY_GROUP**: Level 2 client categorization.
- **MANUFACTURER_INDUSTRY**: Level 3 client categorization.
- **MANUFACTURER_SUBINDUSTRY**: Level 4 client categorization.

In [9]:
class Task:
    def __init__(self, **kwargs):
        """
        A class used to represent a Task. It initializes the attributes dynamically 
        using the keyword arguments passed. Default values are provided for certain fields.
        """
        self.PROJECT_ID = kwargs.get('PROJECT_ID', None)
        self.TASK_ID = kwargs.get('TASK_ID', None)
        self.ASSIGNED = kwargs.get('ASSIGNED', None)
        self.END = kwargs.get('END', None)
        self.SELLING_HOURLY_PRICE = kwargs.get('SELLING_HOURLY_PRICE', None)
        self.MIN_QUALITY = kwargs.get('MIN_QUALITY', None)
        self.WILDCARD = kwargs.get('WILDCARD', None) 
        self.TASK_TYPE = kwargs.get('TASK_TYPE', None)
        self.SOURCE_LANG = kwargs.get('SOURCE_LANG', None)
        self.TARGET_LANG = kwargs.get('TARGET_LANG', None)
        self.MANUFACTURER = kwargs.get('MANUFACTURER', None)
        self.MANUFACTURER_SECTOR = kwargs.get('MANUFACTURER_SECTOR', None)
        self.MANUFACTURER_INDUSTRY_GROUP = kwargs.get('MANUFACTURER_INDUSTRY_GROUP', None)
        self.MANUFACTURER_INDUSTRY = kwargs.get('MANUFACTURER_INDUSTRY', None)
        self.MANUFACTURER_SUBINDUSTRY = kwargs.get('MANUFACTURER_SUBINDUSTRY', None)
        
        # Optional attributes with None default value
        self.START = kwargs.get('START', None)
        self.PM = kwargs.get('PM', None)
        self.TRANSLATOR = kwargs.get('TRANSLATOR', None)
        self.READY = kwargs.get('READY', None)
        self.WORKING = kwargs.get('WORKING', None)
        self.DELIVERED = kwargs.get('DELIVERED', None)
        self.RECEIVED = kwargs.get('RECEIVED', None)
        self.CLOSE = kwargs.get('CLOSE', None)
        self.FORECAST = kwargs.get('FORECAST', None)
        self.HOURLY_RATE = kwargs.get('HOURLY_RATE', None)
        self.COST = kwargs.get('COST', None)
        self.QUALITY_EVALUATION = kwargs.get('QUALITY_EVALUATION', None)
        
    
    def __str__(self):
        return (
            f"Task Details:\n"
            f"  - Task ID: {self.TASK_ID}\n"
            f"  - Type: {self.TASK_TYPE}\n"
            f"  - Client: {self.MANUFACTURER}\n"
            f"  - Sector: {self.MANUFACTURER_SECTOR}\n"
            f"  - Industry (Subsector): {self.MANUFACTURER_INDUSTRY}\n"
            f"  - Start: {self.START}\n"
            f"  - Budget: {self.SELLING_HOURLY_PRICE}\n"
            f"  - Quality: {self.MIN_QUALITY}\n"
            f"  - Wildcard: {self.WILDCARD}\n"
            f"  - Source Language: {self.SOURCE_LANG}\n"
            f"  - Target Language: {self.TARGET_LANG}"
        )

### Split Data

In [10]:
# Split into train and validation (e.g., 80% train, 20% validation)
train_df, validation_df = train_test_split(data_df, test_size=0.2, random_state=42)

In [11]:
def drop_and_save_translator_labels(df, translator_column="TRANSLATOR"):
    """
    Extracts and removes translators from test_df, then saves them as a label dict.
    
    Args:
        df (pd.DataFrame): The dataframe containing translator data.
        translator_column (str): The column name that holds the translator labels.
        
    Returns:
        pd.DataFrame: The test_df without the translators column.
        dict: Dictionary of translator labels {index: translators}
    """
    if translator_column not in df.columns:
        raise ValueError(f"Column '{translator_column}' not found in test_df.")
    
    # Extract labels
    translator_labels = df[translator_column].to_dict()
    
    # Drop the column from the DataFrame
    df = df.drop(columns=[translator_column])
    
    return df, translator_labels

train_df_clean, train_translator_labels = drop_and_save_translator_labels(train_df)
validation_df_clean, validation_translator_labels = drop_and_save_translator_labels(validation_df)

In [12]:
train_df_clean.head()

,PROJECT_ID,PM,TASK_ID,START,END,TASK_TYPE,SOURCE_LANG,TARGET_LANG,ASSIGNED,READY,WORKING,DELIVERED,RECEIVED,CLOSE,FORECAST,HOURLY_RATE,COST,QUALITY_EVALUATION,MANUFACTURER,MANUFACTURER_SECTOR,MANUFACTURER_INDUSTRY_GROUP,MANUFACTURER_INDUSTRY,MANUFACTURER_SUBINDUSTRY
106193,212576,PMT,10313113,2014-11-06 15:29:00,2014-11-10 09:00:00,Translation,English,Spanish (LA),2014-11-06 15:39:16,2014-11-06 15:39:18,2014-11-06 23:18:08,2014-11-10 00:00:44,2014-11-10 12:17:09,2014-11-10 12:17:09,5.78,13,75.14,5,Sparklight Media,Communication Services,Media,Interactive Media & Services,Interactive Media & Services
153803,214379,RMT,10419128,2016-01-25 16:00:00,2016-01-26 12:00:00,ProofReading,English,Catalan,2016-01-25 12:12:58,2016-01-25 12:57:40,2016-01-25 15:35:55,2016-01-25 16:12:16,2016-01-25 16:17:35,2016-01-25 16:17:35,0.29,15,4.35,5,TrueConnect,Communication Services,Interactive Media & Services,Internet Services & Infrastructure,Internet Services & Infrastructure
283706,217532,KMT,10694449,2019-01-07 18:50:00,2019-01-07 21:00:00,Translation,English,Portuguese (Brazil),2019-01-07 19:12:29,2019-01-07 19:12:57,2019-01-07 19:14:00,2019-01-07 19:16:03,2019-01-07 21:07:58,2019-01-07 21:08:04,0.03,15,0.45,10,SoftEcology,Information Technology,Software,Application Software,Environmental Software
493944,221084,KMT,11109570,2022-02-24 15:58:00,2022-02-24 16:55:00,ProofReading,English,Spanish (LA),2022-02-24 15:20:31,2022-02-24 15:57:26,2022-02-24 16:08:34,2022-02-24 16:20:36,2022-02-24 16:49:31,2022-02-24 16:55:10,0.00,14,0.00,6,HealthyLife,Health Care,Health Care Providers,Health Care Facilities,Long-Term Care Facilities
171666,214396,PMT,10455761,2016-07-04 15:34:00,2016-07-06 11:00:00,Translation,English,Spanish (Iberian),2016-07-04 15:41:18,2016-07-04 15:43:01,2016-07-05 09:44:51,2016-07-06 11:04:49,2016-07-08 09:50:06,2016-07-08 09:50:06,4.33,16,69.28,5,WoodWorks,Consumer Discretionary,Leisure Products,Leisure Products,Leisure Products


In [13]:
for i, (key, value) in enumerate(validation_translator_labels.items()):
    print(f"Task Index: {key}, Translator: {value}")
    if i == 10:
        break

Task Index: 89970, Translator: Isaias Venancio
Task Index: 132154, Translator: Xoana
Task Index: 544858, Translator: Mariano Fidel
Task Index: 36852, Translator: Lucia
Task Index: 280522, Translator: Victorino Salvio
Task Index: 59640, Translator: Laurina Rafael
Task Index: 133792, Translator: Oscar
Task Index: 44845, Translator: Andres
Task Index: 252248, Translator: Laurina Rafael
Task Index: 111992, Translator: Dimas Rafael
Task Index: 129220, Translator: Hildegarda Leonor


### Explanation of what is happening

In the global scope:

0. We use as base of useful features the dataframe `transl_cost_pairs_df`
It includes:
- TRANSLATOR: Translator name.
- SOURCE_LANG: Source language.
- TARGET_LANG: Target language.
- HOURLY_RATE: Cost per hour.


1. We compute the **average proportional delay** (speed) from the `data_df` for each translator and merge it with this base dataframe


Now for the each task:

2. We start to the strict filter:
    - Filter of **languages**: only consider the translators who offer this translation
    - Filter of **price**: only consider the prices below the threshold
    - Filter of **quality** by language: this is done by making an average of the quality of these languages for each translator, and then using it as a threshold. 
    - Filter of **availability**: if the taks lasts less than 7 days we check whether the translator will even work before the theoretical deadline (it is 7 days because everyone works at least once a week)

3. We do a **weighted knn**:
    - We do it on the *perfect point* (price = 0, quality = 10, speed = 100%, experience = 10... (orientative values))
    - The weights are chosen by the wildcards and by the expereince required for the type of task. A weighted knn, after normalizing, it distorts the chosen axis size to give more or less weight. 
    - The similarity score is the final ranking. 

4. Outcome possibilities: 
    - We get None or too few translators: if it is None we use the wildcard to completely ignore that factor in the strict filter
    - We get a lot of translators: (we need the rest of features to give better recommendations)

5. Calculate accuracy and find a way to retrieve explainations
    

In [14]:
# ----- THESE ARE GENERAL FUNCTIONS THAT CAN BE USED TO ACTUALIZE THE DATA IF NEW TASKS WERE ADDED TO THE DATASET -----
def compute_delay_percentage(data_df, transl_cost_pairs_df):
    """
    Compute the delay percentage of each translator based on task completion times.
    Negative values indicate early delivery, positive values indicate late delivery.

    Args:
        data_df (pd.DataFrame): Task data with 'TRANSLATOR', 'START', 'END', 'DELIVERED'
        transl_cost_pairs_df (pd.DataFrame): Translator costs with 'TRANSLATOR', 'COST'

    Returns:
        pd.DataFrame: Merged dataframe with 'TRANSLATOR', 'COST', 'AVG_DELAY_PERCENTAGE'
    """
    try:
        # Convert date columns to datetime safely
        date_cols = ['START', 'END', 'DELIVERED']
        for col in date_cols:
            data_df[col] = pd.to_datetime(data_df[col], errors='coerce')

        # Calculate duration and filter out bad rows
        duration = data_df['END'] - data_df['START']
        invalid_rows = duration <= pd.Timedelta(0)
        if invalid_rows.any():
            print(f"[INFO] Removed {invalid_rows.sum()} rows with zero or negative durations.")
            data_df = data_df[~invalid_rows]
            duration = duration[~invalid_rows]

        # Calculate delay percentage
        delay = (data_df['DELIVERED'] - data_df['END']) / duration * 100
        delay = delay.replace([np.inf, -np.inf, np.nan], 0).clip(-100, 100)
        data_df = data_df.copy()
        data_df['DELAY_PERCENTAGE'] = delay

        # Compute average delay per translator
        avg_delay = (
            data_df.groupby('TRANSLATOR')['DELAY_PERCENTAGE']
            .mean()
            .round(2)
            .reset_index()
            .rename(columns={'DELAY_PERCENTAGE': 'AVG_DELAY_PERCENTAGE'})
        )

        # Merge with cost data
        merged = transl_cost_pairs_df.merge(avg_delay, on='TRANSLATOR', how='left')
        merged['AVG_DELAY_PERCENTAGE'] = merged['AVG_DELAY_PERCENTAGE'].fillna(0)

        return merged

    except Exception as e:
        print(f"[ERROR] Failed to compute delay percentages: {e}")
        return transl_cost_pairs_df.assign(AVG_DELAY_PERCENTAGE=0)


def compute_number_tasks(data_df, translators_attributes_df):
    """
    Computes the number of tasks for each translator.
    
    Args:
        data_df (pd.DataFrame): 
            DataFrame containing the data of the tasks.
        df_filtered (pd.DataFrame): 
            DataFrame containing the filtered translators' attributes.

    Returns:
        translators_attributes_df (pd.DataFrame) with the delay_percentage.
    """
    #TAKE INTO ACCOUNT: Task count is added to the dataset to help judge reliability (e.g if the quality of translator is calculated using a few tasks). 

    # Count the number of tasks each translator has done
    task_counts = data_df.groupby('TRANSLATOR').size().reset_index(name='NUM_TASKS')

    # Merge the task counts into the filtered dataframe
    translators_attributes_df = translators_attributes_df.merge(task_counts, on='TRANSLATOR', how='left')

    # Fill missing values (i.e., translators with no tasks) with 0
    translators_attributes_df['NUM_TASKS'] = translators_attributes_df['NUM_TASKS'].fillna(0).astype(int)

    return translators_attributes_df

In [32]:
# ----- THESE ARE FUNCTIONS TO CALCULATE QUALITY AND EXPERIENCE (ONCE FILTERED BY LANGUAGES AND PRICE) -----
def compute_quality_by_languages(data_df, df_filtered, source_lang, target_lang):
    """
    Computes average quality for a given language pair (source_lang → target_lang).
    If the translator has no experience with that task, falls back to:
      - their overall average quality (with a penalty), or
      - a 5 if no task has been done.
    
    Args:
        df_filtered (pd.DataFrame): Filtered translators.
        source_lang (str): Source language.
        target_lang (str): Target language.
    
    Returns:
        pd.DataFrame: Same df_filtered with new 'AVG_QUALITY_BY_LG' column.
    """
    if df_filtered.empty:
        print(f"Warning: No translators found in the filtered dataframe.")
        return df_filtered
    
    translators = df_filtered['TRANSLATOR'].unique()
    
    # Filter tasks dataframe by the language pair and translators in df_filtered
    mask_lang_pair = (
        (data_df['SOURCE_LANG'] == source_lang) &
        (data_df['TARGET_LANG'] == target_lang) &
        (data_df['TRANSLATOR'].isin(df_filtered['TRANSLATOR']))
    )


    # Compute the average quality for each translator in the filtered dataframe
    avg_quality = (
        data_df[mask_lang_pair]
        .groupby('TRANSLATOR')['QUALITY_EVALUATION']
        .mean()
        .round(2)
    )

    # Assing the average quality to the filtered df
    df_filtered['AVG_QUALITY_BY_LG'] = df_filtered['TRANSLATOR'].map(avg_quality)

    # Fallback to penalized overall average
    mask_missing = df_filtered['AVG_QUALITY_BY_LG'].isna()

    overall_avg = (
        data_df[data_df['TRANSLATOR'].isin(translators)]
        .groupby('TRANSLATOR')['QUALITY_EVALUATION']
        .mean()
        .round(2)
        .apply(lambda x: x - 1 if pd.notnull(x) else None)  # configurable penalization, for flexibility
        #For a data-driven approach, use can standard deviation or percentile-based penalization to adapt to the distribution of quality scores
    )

    df_filtered.loc[mask_missing, 'AVG_QUALITY_BY_LG'] = df_filtered.loc[mask_missing, 'TRANSLATOR'].map(overall_avg)

    # Para los traductores que no existen en data_df → asignar calidad por defecto (ej. 5)
    df_filtered['AVG_QUALITY_BY_LG'] = df_filtered['AVG_QUALITY_BY_LG'].fillna(5)
        
    return df_filtered


def compute_quality_by_task_type(data_df, df_filtered, task_type):
    """
    Computes the average quality for each translator for a given task type.
    If the translator has no experience with that task, falls back to:
      - their overall average quality (with a penalty), or
      - a 5 if no task has been done.
    
    Args:
        df_filtered (pd.DataFrame): DataFrame with filtered translators.
        task_type (str): The specific task type to evaluate.
    
    Returns:
        pd.DataFrame: df_filtered with 'AVG_QUALITY_BY_TASK' and 'QUALITY_SOURCE_TASK'.
    """
    if df_filtered.empty:
        print(f"Warning: No translators found in the filtered dataframe.")
        return df_filtered
    
    translators = df_filtered['TRANSLATOR'].unique()

    # 1. Compute average quality for given task type
    mask_task = (
        (data_df['TASK_TYPE'] == task_type) &
        (data_df['TRANSLATOR'].isin(translators))
    )

    avg_by_task = (
        data_df[mask_task]
        .groupby('TRANSLATOR')['QUALITY_EVALUATION']
        .mean()
        .round(2)
    )

    df_filtered['AVG_QUALITY_BY_TASK'] = df_filtered['TRANSLATOR'].map(avg_by_task)

    # 2. Fallback to penalized overall average
    mask_missing = df_filtered['AVG_QUALITY_BY_TASK'].isna()

    overall_avg = (
        data_df[data_df['TRANSLATOR'].isin(translators)]
        .groupby('TRANSLATOR')['QUALITY_EVALUATION']
        .mean()
        .round(2)
        .apply(lambda x: x - 1 if pd.notnull(x) else None)  # configurable penalization, for flexibility
        #For a data-driven approach, use can standard deviation or percentile-based penalization to adapt to the distribution of quality scores
    )

    df_filtered.loc[mask_missing, 'AVG_QUALITY_BY_TASK'] = df_filtered.loc[mask_missing, 'TRANSLATOR'].map(overall_avg)

    df_filtered['AVG_QUALITY_BY_TASK'] = df_filtered['AVG_QUALITY_BY_TASK'].fillna(5)

    return df_filtered


def compute_experience(df_filtered, task_type, source_lang, target_lang, industry, subindustry):
    """
    Computes a soft experience score for each translator based on how many
    dimensions match (task_type, language pair, industry, subindustry).

    Args:
        df_filtered (pd.DataFrame): Filtered translators' dataframe.

    Returns:
        pd.DataFrame: With added column 'EXPERIENCE_SCORE'.
    """
    TASK_TYPE_BONUS = {
        'LanguageLead': 0.5,
        'ProofReading': 0.5,
        'Spotcheck': 0.5
    }

    translators = df_filtered['TRANSLATOR'].unique()

    df = data_df[data_df['TRANSLATOR'].isin(translators)].copy()

    # Base score: match on source, target, task_type
    df['score'] = 0
    df['score'] += (df['SOURCE_LANG'] == source_lang).astype(int)
    df['score'] += (df['TARGET_LANG'] == target_lang).astype(int)
    df['score'] += (df['TASK_TYPE'] == task_type).astype(int)

    # Only add 1 point if industry or subindustry match
    industry_match = (df['MANUFACTURER_INDUSTRY'] == industry)
    subindustry_match = (df['MANUFACTURER_SUBINDUSTRY'] == subindustry)
    df['score'] += ((industry_match | subindustry_match)).astype(int)

    # Advanced task bonus
    bonus_df = df[df['TASK_TYPE'].isin(TASK_TYPE_BONUS)].copy()
    bonus_df['bonus'] = bonus_df['TASK_TYPE'].map(TASK_TYPE_BONUS)
    bonus_scores = bonus_df.groupby('TRANSLATOR')['bonus'].sum()

    # Base score
    base_scores = df.groupby('TRANSLATOR')['score'].sum()

    # Total experience = base + bonus
    total_score = base_scores.add(bonus_scores, fill_value=0)

    df_filtered['EXPERIENCE_SCORE'] = df_filtered['TRANSLATOR'].map(total_score).fillna(0)

    # Normalize between 0 and 10
    min_score = df_filtered['EXPERIENCE_SCORE'].min()
    max_score = df_filtered['EXPERIENCE_SCORE'].max()

    if max_score > min_score:
        df_filtered['EXPERIENCE_SCORE'] = (
            (df_filtered['EXPERIENCE_SCORE'] - min_score) / (max_score - min_score)
        ) * 10
    else:
        df_filtered['EXPERIENCE_SCORE'] = 0

    df_filtered['EXPERIENCE_SCORE'] = df_filtered['EXPERIENCE_SCORE'].round(2)

    # Detect translators not present in data_df (no prior tasks)
    missing_translators_mask = ~df_filtered['TRANSLATOR'].isin(data_df['TRANSLATOR'])
    missing_translators = df_filtered.loc[missing_translators_mask, 'TRANSLATOR']

    # if not missing_translators.empty:
    #     print("Translators with no experience data:")
    #     print(missing_translators.tolist())

    # Compute average from those with scores
    avg_experience = df_filtered.loc[~missing_translators_mask, 'EXPERIENCE_SCORE'].mean()
    # print(f"Assigning average experience score of {round(avg_experience, 2)} to missing translators.")

    # Assign average to missing
    df_filtered.loc[missing_translators_mask, 'EXPERIENCE_SCORE'] = avg_experience

    return df_filtered


def compute_experience_for_client(df_filtered, client):
    """
    Computes an experience score for each translator based on a specific client

    Args:
        df_filtered (pd.DataFrame): Filtered translators' dataframe.

    Returns:
        pd.DataFrame: With added column 'EXPERIENCE_CLIENT'.
    """
    translators = df_filtered['TRANSLATOR'].unique()
    df = data_df[data_df['TRANSLATOR'].isin(translators)].copy()


    df['score'] = 0
    df['score'] += (df['MANUFACTURER'] == client).astype(int)

    # Total experience score = sum of weights per translator
    experience_scores = df.groupby('TRANSLATOR')['score'].sum()

    # Add to filtered dataframe
    df_filtered['EXPERIENCE_CLIENT'] = df_filtered['TRANSLATOR'].map(experience_scores).fillna(0).astype(int)

    # Normalizar entre 0 y 10
    min_score = df_filtered['EXPERIENCE_CLIENT'].min()
    max_score = df_filtered['EXPERIENCE_CLIENT'].max()

    if max_score > min_score:  # Evitar división por 0
        df_filtered['EXPERIENCE_CLIENT'] = ((df_filtered['EXPERIENCE_CLIENT'] - min_score) / (max_score - min_score)) * 10
    else:
        df_filtered['EXPERIENCE_CLIENT'] = 0  # Si todos los scores son iguales

    df_filtered['EXPERIENCE_CLIENT'] = df_filtered['EXPERIENCE_CLIENT'].round(2)

    return df_filtered


def available_translators(task, translators_attributes_df, schedules_df, TRANSLATORS_UNAVAILABLE):
    """
    Checks if translators are available for the task based on their weekly working schedule.
    This, for now just takes into account the day of the week and the start time of the task. 
    TAKE INTO ACCOUNT: This can have problems if the translator is at the end of their weekly shedule, it also doesnt take into account multitasking.
    
    Args:
        task (Task object): The task for which we want to check availability.
        translators_attributes_df (pd.DataFrame): DataFrame containing the translators' attributes.
        schedules_df (pd.DataFrame): DataFrame containing the weekly schedules of translators.
        TRANSLATORS_UNAVAILABLE (list): List of translators who are unavailable.
        
    Returns:
        df_filtered (pd.DataFrame): Filtered DataFrame containing translators who are available.
    """
    ##TODO:check multitasking
    # 1. Remove explicitly unavailable translators
    df_filtered = translators_attributes_df[~translators_attributes_df['TRANSLATOR'].isin(TRANSLATORS_UNAVAILABLE)].copy()

    # 2. Extract day of week and time from task
    task_day = task.ASSIGNED.strftime('%a').upper()  #day of the week  e.g., 'MON', 'TUE'

    # 3. Merge schedule info
    df_filtered = df_filtered.merge(schedules_df, left_on='TRANSLATOR', right_on='NAME').drop(columns=['NAME'])

    def is_available(row):
        # 3.1 Check if works that day
        if row[task_day] != 1:
            return False

        # 3.2 Parse working hours
        task_start_time = timedelta(hours=task.ASSIGNED.hour, minutes=task.ASSIGNED.minute)
        task_end_time = timedelta(hours=task.ASSIGNED.hour +1, minutes=task.ASSIGNED.minute) # Assuming 1 hour task duration 
        ## TODO change this to the estimated duration of the task based on std and mean of the task type

        work_start_time = timedelta(hours=row['START'].hour, minutes=row['START'].minute)
        work_end_time = timedelta(hours=row['END'].hour, minutes=row['END'].minute)
        
        return (task_start_time >= work_start_time) and (task_end_time < work_end_time) 

    # 4. Apply availability logic
    df_filtered['IS_AVAILABLE'] = df_filtered.apply(is_available, axis=1)
    df_filtered = df_filtered[df_filtered['IS_AVAILABLE'] == True].drop(columns=['IS_AVAILABLE'])

    return df_filtered


# ----- PRINCIPAL FUNCTION TO FILTER THE TRANSLATORS' ATTRIBUTES -----
def filter_language_price_quality_availability(data_df, schedules_df, translators_attributes_df, task = Task, need_wildcard = False):
    """
    Filters the translators' attributes by languages, price, quality and availability.
    If need_wildcard is True, it will skip the filter corresponding to the wildcard.

    Structured fallback:
        Tries a strict filter.
        If that fails, it retries with a wildcard (skipping one constraint).
        If that also fails, it relaxes all filters except language pair. That's reasonable.
    
    Args:
        translators_attributes_df (pd.DataFrame): 
            DataFrame containing the translators' attributes (name, languages, price, speed).
        task (Task object): 
            The task for which we want to filter the translators.
        need_wildcard (bool): 
            If True, skip the filter corresponding to the wildcard.
            
    Returns:
        pd.DataFrame: 
            Filtered DataFrame containing translators who meet the criteria.
    """
    
    if not need_wildcard:
        # Filter by language, price HARD FILTER
        df_filtered = translators_attributes_df[
            (translators_attributes_df['SOURCE_LANG'] == task.SOURCE_LANG) & 
            (translators_attributes_df['TARGET_LANG'] == task.TARGET_LANG) &
            (translators_attributes_df['HOURLY_RATE'] <= task.SELLING_HOURLY_PRICE) 
        ].copy()

        # If the filtered dataframe is empty, it's because the budget is too low, therefore we have to relax the filter
        if df_filtered.empty:
            if task.WILDCARD == "Price":
                print(f"Warning: No translators found for task {task.TASK_ID}, wildcard= {task.WILDCARD} because the BUDGET is too low. Trying with wildcard...")
                return filter_language_price_quality_availability(data_df, schedules_df, translators_attributes_df, task = task, need_wildcard = True)
            else: 
                print(f"Warning: No translators found for task {task.TASK_ID}, wildcard= {task.WILDCARD} because the BUDGET is too low. Relaxing price filter...")
                # Skip the price filter and try again
                df_filtered = translators_attributes_df[
                    (translators_attributes_df['SOURCE_LANG'] == task.SOURCE_LANG) &
                    (translators_attributes_df['TARGET_LANG'] == task.TARGET_LANG)
                ].copy()

        # add the average quality column
        df_filtered = compute_quality_by_task_type(data_df, df_filtered, task_type=task.TASK_TYPE)
        df_filtered = compute_quality_by_languages(data_df, df_filtered, source_lang=task.SOURCE_LANG, target_lang=task.TARGET_LANG)

        df_filtered_quality = df_filtered[df_filtered['AVG_QUALITY_BY_LG'] >= task.MIN_QUALITY]

        # If the filtered dataframe is empty, it's because the quality is too high.
        if df_filtered_quality.empty:
            if task.WILDCARD == "Quality":
                print(f"Warning: No translators found for task {task.TASK_ID}, wildcard= {task.WILDCARD} because the QUALITY is too high. Trying with wildcard...")
                return filter_language_price_quality_availability(data_df, schedules_df, translators_attributes_df, task = task, need_wildcard = True)
            else: 
                print(f"Warning: No translators found for task {task.TASK_ID}, wildcard= {task.WILDCARD} because the QUALITY is too high. Relaxing quality filter...")
                # Skip the quality filter and try again
                df_filtered_quality = df_filtered[df_filtered['AVG_QUALITY_BY_LG'] >= task.MIN_QUALITY-2] # Relaxing the quality filter by 2 points
            
        
        # Filter by availability
        df_filtered_availability = available_translators(task, df_filtered_quality, schedules_df, TRANSLATORS_UNAVAILABLE)

        if df_filtered_availability.empty:
            if task.WILDCARD == "Deadline":
                print(f"Warning: No translators found for task {task.TASK_ID}, wildcard= {task.WILDCARD} because TRANSLATORS ARE UNAVAILABLE. Trying with wildcard...")
                return filter_language_price_quality_availability(data_df, schedules_df, translators_attributes_df, task = task, need_wildcard = True)
            else: 
                print(f"Warning: No translators found for task {task.TASK_ID}, wildcard= {task.WILDCARD} because TRANSLATORS ARE UNAVAILABLE. Relaxing availability filter...")
                return df_filtered_quality

        return df_filtered_availability
    
    # same code as above but with the wildcard, it will skip the filter corresponding to the wildcard
    else:
        # if the wildcard is "Price", we don't filter by price
        price_condition = (translators_attributes_df['HOURLY_RATE'] <= task.SELLING_HOURLY_PRICE) if task.WILDCARD != "Price" else True
        # Filter by language, price 
        df_filtered = translators_attributes_df[
            (translators_attributes_df['SOURCE_LANG'] == task.SOURCE_LANG) & 
            (translators_attributes_df['TARGET_LANG'] == task.TARGET_LANG) &
            price_condition 
        ].copy()

        if df_filtered.empty:
            print(f"Warning: No translators found for task {task.TASK_ID}, wildcard= {task.WILDCARD} because the BUDGET is too low. Relaxing price filter...")
            # Skip the price filter and try again
            df_filtered = translators_attributes_df[
                (translators_attributes_df['SOURCE_LANG'] == task.SOURCE_LANG) &
                (translators_attributes_df['TARGET_LANG'] == task.TARGET_LANG)
            ].copy()

        # add the average quality column
        df_filtered = compute_quality_by_languages(data_df, df_filtered, source_lang=task.SOURCE_LANG, target_lang=task.TARGET_LANG)
        df_filtered = compute_quality_by_task_type(data_df, df_filtered, task_type=task.TASK_TYPE)

        if task.WILDCARD != "Quality":
            df_filtered_quality = df_filtered[df_filtered['AVG_QUALITY_BY_LG'] >= task.MIN_QUALITY]

            if df_filtered_quality.empty:
                print(f"Warning: No translators found for task {task.TASK_ID}, wildcard= {task.WILDCARD} because the QUALITY is too high. Relaxing quality filter...")
                # Skip the quality filter and try again
                df_filtered_quality = df_filtered[df_filtered['AVG_QUALITY_BY_LG'] >= task.MIN_QUALITY-2] # Relaxing the quality filter by 2 points
        else:
            df_filtered_quality = df_filtered.copy()

        if task.WILDCARD != "Deadline":  
            # Filter by availability
            df_filtered_availability = available_translators(task, df_filtered_quality, schedules_df, TRANSLATORS_UNAVAILABLE)

            if df_filtered_availability.empty:
                print(f"Warning: No translators found for task {task.TASK_ID}, wildcard= {task.WILDCARD} because TRANSLATORS ARE UNAVAILABLE. Relaxing availability filter...")
                return df_filtered_quality
        
        return df_filtered

In [16]:
"""
At this stage, we have a filtered dataframe with the translators that are available for the task,
it contains the following columns:
    - TRANSLATOR: Name of the translator
    - SOURCE_LANG: Source language of the translator
    - TARGET_LANG: Target language of the translator
    - HOURLY_RATE: Hourly rate of the translator
    - Filtered by the availability

    Things to take into account for the calculation of the scores:
    - AVG_QUALITY_BY_LNG: Average quality by language pair (if applicable)
    - AVG_QUALITY_BY_TASK: Average quality by task type (if applicable)
    - QUALITY_SOURCE_TASK: Source of the quality by task score (original, overall_penalized, global_penalized)
    - NUM_TASKS: Number of tasks performed by the translator (to take into account the reliability of the translator's quality and delay percentage, not experience because it is calculated based on specific tasks)

    - EXPERIENCE_SCORE: Experience score based on task type, language pair, industry, and subindustry
    - EXPERIENCE_CLIENT: Experience score based on the specific client (if applicable)
    - AVG_DELAY_PERCENTAGE: Average delay percentage of the translator (if applicable)

    
There are some key considerations regarding the experience and quality weights:
    - Proofreading and Spotcheck need more expereinced translators.
    - LanguageLead is a more advanced task, so it needs more experience and quality.
    - Test should be assigned to the most experienced and high-quality TRANSLATOR for the client or topic, regardless of price.
    - For training we dont need to take into account the experience nor the quality

"""

"\nAt this stage, we have a filtered dataframe with the translators that are available for the task,\nit contains the following columns:\n    - TRANSLATOR: Name of the translator\n    - SOURCE_LANG: Source language of the translator\n    - TARGET_LANG: Target language of the translator\n    - HOURLY_RATE: Hourly rate of the translator\n    - Filtered by the availability\n\n    Things to take into account for the calculation of the scores:\n    - AVG_QUALITY_BY_LNG: Average quality by language pair (if applicable)\n    - AVG_QUALITY_BY_TASK: Average quality by task type (if applicable)\n    - QUALITY_SOURCE_TASK: Source of the quality by task score (original, overall_penalized, global_penalized)\n    - NUM_TASKS: Number of tasks performed by the translator (to take into account the reliability of the translator's quality and delay percentage, not experience because it is calculated based on specific tasks)\n\n    - EXPERIENCE_SCORE: Experience score based on task type, language pair, in

In [17]:
# ----- KNN ----
def knn(df_filtered, task, metric = 'euclidean', need_wildcard=False):
    """
    Optimized KNN to find the best translators based on the task's requirements.
    
    Args:
        df_filtered (pd.DataFrame): DataFrame containing the filtered translators' attributes.
        task (Task object): The task for which we are finding suitable translators.
        need_wildcard (bool): Whether to ignore the wildcard feature in KNN calculation.

    Returns:
        distances (np.ndarray): Distances of the nearest neighbors.
        indexes (np.ndarray): Indices of the nearest neighbors in the original DataFrame.
    """
    # Define features for the KNN
    features = ['HOURLY_RATE', 'AVG_QUALITY_BY_LG', 'AVG_QUALITY_BY_TASK', 'AVG_DELAY_PERCENTAGE', 'EXPERIENCE_SCORE', 'EXPERIENCE_CLIENT']
    ideal_values = [1, 10, 10, -100, 10, 10]  # Ideal values for the features (price, quality, speed, experience), this is like the ideal translator
    
    # Initialize weights (weights for each feature)
    weights = np.array([1, 1.5, 1.5, 0.25, 1, 0.5])  # Default weights for the features

    # Adjust weights based on task type
    if task.TASK_TYPE == 'ProofReading' or task.TASK_TYPE == 'Spotcheck':
        # Higher weight for experience and quality
        weights[4] *= 2  # Experience score weight increased

    elif task.TASK_TYPE == 'LanguageLead':
        # LanguageLead requires more experience and quality
        weights[4] *= 2
        weights[1] *= 2
        weights[2] *= 2

    elif task.TASK_TYPE == 'Test':
        # Test task needs the highest experience and quality, ignore price
        weights[0] = 0  # Price doesn't matter for Test
        weights[1] *= 2  # Quality by language pair weight increased
        weights[2] *= 2  # Quality by task weight increased
        weights[4] *= 2  # Experience score weight increased
        weights[3] *= 3  # Experience score  for client weight increased

    elif task.TASK_TYPE == 'Training':
        # Training task doesn't consider experience or quality
        weights[1] = 0  # Set quality to 0
        weights[2] = 0  # Set quality to 0
        weights[3] = 0  # Set experience to 0
        weights[4] = 0  # Set experience to 0

    # If wildcard is required, adjust the weights based on the wildcard preference
    if not need_wildcard:
        wildcard_vector = np.ones_like(weights)

        if task.WILDCARD == 'Price':
            wildcard_vector[0] = 0.25  # Price becomes less important
        elif task.WILDCARD == 'Quality':
            wildcard_vector[1] = 0.25  # Quality becomes less important
        elif task.WILDCARD == 'Deadline':
            wildcard_vector[3] = 0.25  # Deadline (Avg. Delay) becomes less important

        weights = weights * wildcard_vector  # Apply adjusted weights based on wildcard

    # Select features for the KNN calculation
    X = df_filtered[features]

    # Standardize the features (scaling)
    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X)
    X_weighted = X_scaled * weights  # Apply the weights to scaled features

    # Initialize and train the KNN model
    knn = NearestNeighbors(metric=metric)
    knn.fit(X_weighted)

    # Define ideal task values as a DataFrame
    task_df = pd.DataFrame([ideal_values], columns=features)
    task_scaled = scaler.transform(task_df)
    task_weighted = task_scaled * weights  # Weight the task ideal values too

    # Find the nearest neighbors based on the task's ideal values
    distances, indexes = knn.kneighbors(task_weighted, n_neighbors=len(df_filtered))  # Adjust the number of neighbors as needed

    return distances, indexes


def get_best_translators(df_filtered, indexes, distances):
    """
    Get the best translators based on the KNN results.
    
    Args:
        df_filtered (pd.DataFrame): 
            Contains the filtered translators' attributes (name, language, price, quality, speed).
        indexes (np.ndarray): 
            Indices of the nearest neighbors in the df_filtered.
        distances (np.ndarray): 
            Distances of the nearest neighbors.
            
    Returns:
        df_filtered (pd.DataFrame): 
            Contains the filtered translators' attributes (name, language, price, quality, speed AND similarity_score).
    """
    
    selected_translators = df_filtered.iloc[indexes[0]].copy()
    
    # Add the similarity score
    selected_translators['Similarity Score'] = distances[0].round(2)  # Round to 2 decimal places

    # Sort by similarity score (ascending: closest match first)
    selected_translators = selected_translators.sort_values(by='Similarity Score', ascending=False) 

    return selected_translators

In [18]:
def evaluate_metrics(df_filtered, task):
    metrics = ['euclidean', 'manhattan']
    results = {}

    for metric in metrics:
        distances, indexes = knn(df_filtered, task, metric=metric, need_wildcard=False)
        selected_translators = get_best_translators(df_filtered, indexes, distances)
        
        # Guarda los resultados para cada métrica
        results[metric] = selected_translators
    
    return results


def plot_similarity(results):
    """
    Plot the similarity scores for translators selected using different distance metrics.
    
    Args:
        results (dict): A dictionary where keys are metric names and values are DataFrames 
                        with the selected translators' similarity scores.
    """
    # Create a DataFrame to hold all similarity scores
    similarity_data = []
    for metric, translators in results.items():
        # Add a new column with the metric name
        translators['Metric'] = metric
        similarity_data.append(translators)

    # Concatenate all DataFrames
    all_similarity_scores = pd.concat(similarity_data)

    # Plot using seaborn
    plt.figure(figsize=(10, 6))
    sns.boxplot(x='Metric', y='Similarity Score', data=all_similarity_scores)
    plt.title('Comparison of Similarity Scores for Different Metrics')
    plt.xlabel('Distance Metric')
    plt.ylabel('Similarity Score')
    plt.show()

Cosine similaruty is not considered, as it measures how similarly two vectors point in the same direction, regardless of their magnitude, and is used when relative proportions matter more than absolute values.

In [31]:
# ------------ DEBUGGGING ------------
# Creates a dataframe with  the additional attributes
translators_attributes_df = compute_delay_percentage(train_df, transl_cost_pairs_df)
translators_attributes_df = compute_number_tasks(train_df, translators_attributes_df) 

task_row = validation_df_clean[validation_df_clean['TASK_ID'] == 11175505].iloc[0]
#Convert the current row to a task
new_task = task_row.copy()
new_task = new_task.rename({'HOURLY_RATE': 'SELLING_HOURLY_PRICE', 'QUALITY_EVALUATION': 'MIN_QUALITY'})

match = clients_df[clients_df['CLIENT_NAME'].str.strip() == new_task['MANUFACTURER'].strip()]

if not match.empty:
    new_task['WILDCARD'] = match.iloc[0]['WILDCARD']
    new_task['HOURLY_RATE'] = match.iloc[0]['SELLING_HOURLY_PRICE']
    new_task['QUALITY_EVALUATION'] = match.iloc[0]['MIN_QUALITY']
else:
    print("WARNING: No match found in schedules_df for the given client. Setting default values.")
    new_task['WILDCARD'] = 'Quality'
    new_task['HOURLY_RATE'] = new_task['SELLING_HOURLY_PRICE']
    new_task['QUALITY_EVALUATION'] = new_task['MIN_QUALITY']

new_task = Task(**new_task.to_dict())  # Convert the task to the Task object
print(new_task)
df_filtered = translators_attributes_df[
            (translators_attributes_df['SOURCE_LANG'] == new_task.SOURCE_LANG) & 
            (translators_attributes_df['TARGET_LANG'] == new_task.TARGET_LANG) 
        ].copy()

df_filtered = compute_quality_by_languages(data_df, df_filtered, source_lang=new_task.SOURCE_LANG, target_lang=new_task.TARGET_LANG)
df_filtered = compute_quality_by_task_type(data_df, df_filtered, task_type=new_task.TASK_TYPE)

df_filtered = df_filtered[df_filtered['AVG_QUALITY_BY_LG'] >= new_task.MIN_QUALITY]

df_filtered = available_translators(new_task, df_filtered, schedules_df, TRANSLATORS_UNAVAILABLE)

display(df_filtered)


Task Details:
  - Task ID: 11175505
  - Type: Translation
  - Client: InnoSoft
  - Sector: Information Technology
  - Industry (Subsector): Software
  - Start: 2022-07-27 00:05:00
  - Budget: 13
  - Quality: 4
  - Wildcard: Deadline
  - Source Language: English
  - Target Language: Spanish (Iberian)


,TRANSLATOR,SOURCE_LANG,TARGET_LANG,HOURLY_RATE,AVG_DELAY_PERCENTAGE,NUM_TASKS,AVG_QUALITY_BY_LG,AVG_QUALITY_BY_TASK,START,END,MON,TUE,WED,THU,FRI,SAT,SUN


In [33]:
# ----- MAIN CODE ----
# Creates a dataframe with  the additional attributes
translators_attributes_df = compute_delay_percentage(train_df, transl_cost_pairs_df)
translators_attributes_df = compute_number_tasks(train_df, translators_attributes_df) 

# Define the top-k value for evaluation
k = 10
correct_predictions = 0
total_predictions = 0

# Iterate over each row in the validation set
for idx, task_row in validation_df_clean.iterrows():
    #Convert the current row to a task
    new_task = task_row.copy()
    new_task = new_task.rename({'HOURLY_RATE': 'SELLING_HOURLY_PRICE', 'QUALITY_EVALUATION': 'MIN_QUALITY'})
    
    match = clients_df[clients_df['CLIENT_NAME'].str.strip() == new_task['MANUFACTURER'].strip()]

    if not match.empty:
        new_task['WILDCARD'] = match.iloc[0]['WILDCARD']
        new_task['HOURLY_RATE'] = match.iloc[0]['SELLING_HOURLY_PRICE']
        new_task['QUALITY_EVALUATION'] = match.iloc[0]['MIN_QUALITY']
    else:
        print("WARNING: No match found in schedules_df for the given client. Setting default values.")
        new_task['WILDCARD'] = 'Quality'
        new_task['HOURLY_RATE'] = new_task['SELLING_HOURLY_PRICE']
        new_task['QUALITY_EVALUATION'] = new_task['MIN_QUALITY']
    
    new_task = Task(**new_task.to_dict())  # Convert the task to the Task object
    
    # Filter translators based on task attributes
    df_filtered = filter_language_price_quality_availability(data_df, schedules_df, translators_attributes_df, new_task)

    if df_filtered.empty:
        print(f"No translators found for task {new_task.TASK_ID}, {new_task.SOURCE_LANG} → {new_task.TARGET_LANG} . Skipping...")

    # Compute experience scores for the filtered translators
    compute_experience(df_filtered, task_type=new_task.TASK_TYPE, source_lang=new_task.SOURCE_LANG, target_lang=new_task.TARGET_LANG, industry=new_task.MANUFACTURER_INDUSTRY, subindustry=new_task.MANUFACTURER_SUBINDUSTRY)
    compute_experience_for_client(df_filtered, client=new_task.MANUFACTURER)

    # Get the distances and indexes from KNN
    distances, indexes = knn(df_filtered, new_task)
    
    # Get the best translators
    selected_translators = get_best_translators(df_filtered, indexes, distances)
    
    # Retrieve the true translator label for the current task
    true_translator = validation_translator_labels[idx]  # Assuming you have the true labels in the dictionary from earlier
    
    # Check if any of the top-k translators match the true translator
    top_k_translators = selected_translators.iloc[:k]  # Top-k translators
    if true_translator in top_k_translators['TRANSLATOR'].values:
        correct_predictions += 1
    
    total_predictions += 1

# Calculate the top-k accuracy
top_k_accuracy = correct_predictions / total_predictions
print(f"Top-{k} Accuracy: {top_k_accuracy * 100:.2f}%")

KeyboardInterrupt: 

#### one problem: some of the translators (13) finish after 0am 

### Next steps
actualize the unavailable translators:
- add it to the list when one is selected by the client in the forntend
- remove it from the list when the task has been finished 